# Causes Behind Homicides 


For this project I wanted to tackle the problem of the [United States elevated Homicide Rate](https://www.statista.com/statistics/1374211/g7-country-homicide-rate/) compared to other highly developed nations. Much of the consideration around this topic resolves around Poverty, Gun Ownership, and Mental Health. Specifically, I wanted to look at the these different metrics at a county level using Census and CDC datasources; In the following notebook I take my raw data and perform the proper cleaning and data modfication required before moving to EDA and finally Model Creation. I hope you find this project informative and interesting. 

My DataSources are dervied from these link: 

Population Density (2020): 
https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?location=18.286026%2C0.315550%2C2.18&showTable=true

Suicides, Homicides, and Drug and Alcohol Related Death(2010-2020): https://wonder.cdc.gov/controller/datarequest/

Income & Poverty (2022): https://www.census.gov/programs-surveys/saipe.html

Before diving into the data in order to protect the privacy counties, the CDC will supress data if the count is low enough to track the cause of death back to the deceased. These are some consideration I had to take tackle when working on this project. I will walk through the data cleaning process and finally the model construction step by step over three notebooks. 

First lets import some of the libraries we will be using. 

## Part 1: Data Cleaning

In [2]:
import pandas as pd
import numpy as np

## Homicides
Lets first import and clean our homicide data. While the data can be extracted from the data source linked above I have also provided a csv file for convenience. 
Here we will use pandas to import this csv file, and drop the rows which are not counties. Finally, we will just filter the data too look at the combined Homicide rows. Here we can see there are some 3147 counties, that will be the starting point for our data. 


In [3]:
df_homicides = pd.read_csv('data/Raw Data/CDC_Homicides_2010_2020.csv')
df_homicides = df_homicides.dropna(subset = ['County'])
df_homicides = df_homicides.loc[df_homicides['ICD-10 113 Cause List']
                                =='#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)']

print('Total Counties: ', len(df_homicides)) 

Total Counties:  3147


Next we will remove the counties will 'Suppressed' or 'Missing' data since they are not relevant to our analysis. We can view a few rows of our dataset to get a sense of its structure. Note that the population figures are cumalative over a 10 year period.

In [4]:
df_homicides = df_homicides[(df_homicides['Deaths']!='Suppressed')&(df_homicides['Deaths']!= 'Missing')]

print('Number of Counties with Data: ', len(df_homicides))

#Change column names
df_homicides = df_homicides[['County','Deaths','Population']].rename(columns = {'Deaths':'Homicides'})
df_homicides.head()

Number of Counties with Data:  1926


,County,Homicides,Population
0,"Autauga County, AL",32,609875
3,"Baldwin County, AL",68,2250866
6,"Barbour County, AL",29,287620
9,"Bibb County, AL",21,248120
12,"Blount County, AL",38,635351


## Suicides 

Now we can import the data relating with suicides. This data (also from the CDC) will serve two purposes. First Suicides via Firearm is a well known [proxy of gun ownership](https://pubmed.ncbi.nlm.nih.gov/37160211/). Second, Suicide Rate can be a determination for the status of mental health in the particular county. As we did with Homicide we will first import the data and drop rows have Nans for county.

In [11]:
df_suicides = pd.read_csv('data/Raw Data/CDC_Suicides_2010_2020.csv')
df_suicides = df_suicides.dropna(subset = ['County'])

We will now take the rows that have data regarding Suicides via Firearms and All Suicides. After that we can merge this will our Homicide Data by creating a new DataFrame (df_combined). Notice that here, some of our Suicide Data will be supressed. For now we will just included denote those with the value 'Suppressed'.

In [12]:
df_suicides = df_suicides[['County','Deaths','ICD-10 113 Cause List']]
df_suicides = df_suicides[df_suicides['ICD-10 113 Cause List']== 'All'].merge(df_suicides[df_suicides['ICD-10 113 Cause List'] == 'Firearms'], on = 'County')
df_suicides = df_suicides.rename(columns = {'Deaths_y': 'Firearms', 'Deaths_x': 'All'})[['County','Firearms','All']]
df_suicides.head()

,County,Firearms,All
0,"Autauga County, AL",78,115
1,"Baldwin County, AL",281,444
2,"Barbour County, AL",30,41
3,"Bibb County, AL",29,38
4,"Blount County, AL",83,122


In [13]:
df_combined = df_suicides.merge(df_homicides, on = 'County', how = 'right')
print('Remaining Counties without complete Suicide Data', len(df_combined[df_combined['Firearms']=='Suppressed']))

Remaining Counties without complete Suicide Data 201


## Drug Deaths

Finally, lets pull in our Drug and Alcohol Death related data (another stand in for Mental Health). Once again we will merge the seperate rows on the county level (similiar to a pivot table).

In [16]:
df_drug_raw = pd.read_csv('data/Raw Data/CDC_Drug_Related_2010_2020.csv').dropna(subset='County')

df_drug = df_drug_raw[df_drug_raw['Drug/Alcohol Induced Code']=='D'].merge(df_drug_raw[df_drug_raw['Drug/Alcohol Induced Code']=='A'], on = 'County')
df_drug = df_drug.rename(columns={'Deaths_x': 'Drugs', 'Deaths_y': 'Alcohol'})[['County','Drugs','Alcohol']]
df_drug.head()

,County,Drugs,Alcohol
0,"Autauga County, AL",61,33
1,"Baldwin County, AL",377,237
2,"Barbour County, AL",17,14
3,"Bibb County, AL",47,Suppressed
4,"Blount County, AL",140,48


Now that we are done combining all our data that was supressed we can take a look and see how much of it is missing. Here I decided to drop the rows which had no CDC data coming in since there are of little use in this analysis. We can see however that of the 1926 counties we started with (in Homicide Data) and large majority (1629) of all the CDC data. 

In [17]:
df_combined = df_drug.merge(df_combined, on = 'County', how = 'right')

count_suppressed = len(df_combined.loc[(df_combined['Alcohol'] == 'Suppressed') & (df_combined['Drugs'] == 'Suppressed')])

print('Counties with Supressed Drug Deaths: ' ,len(df_combined[df_combined['Drugs']=='Suppressed']))
print('Counties with Supressed Alcohol Deaths: ' ,len(df_combined[df_combined['Alcohol']=='Suppressed']))
print(f'Counties with Supressed Deaths for Both: {count_suppressed}' )

Counties with Supressed Drug Deaths:  189
Counties with Supressed Alcohol Deaths:  226
Counties with Supressed Deaths for Both: 142


In [18]:
all_supressed = ((df_combined['Alcohol'] == 'Suppressed') &
               (df_combined['Drugs'] == 'Suppressed') &
               (df_combined['Firearms'] == 'Suppressed'))

print('Counties were all CDC data is Suppressed', all_supressed.sum())

df_combined = df_combined[~all_supressed]

Counties were all CDC data is Suppressed 116


In [19]:
count_valid = ((df_combined['Alcohol'] != 'Suppressed') &
               (df_combined['Drugs'] != 'Suppressed') &
               (df_combined['Firearms'] != 'Suppressed')).sum()

print('Counties were all CDC data is not Suppressed', count_valid)


Counties were all CDC data is not Suppressed 1629


## Income 

Poverty has long been demonstrated as a major driver of crime, so naturally it should be one of the variables we examine. I've decided to pull in both overall Poverty % as well as the rate for U18. Later on we will decided which of these two attributes we will use in the final modeling step. Here I have taken the Poverty Data from the US Census. The process of getting the data in requires to first formatting our county names in the same way as it was done in the CDC data, and limiting our analysis to only countines (indicated by 'County FIPS Code'). Below we can see what our restructured dataset looks like.

In [21]:
df_income = pd.read_csv('data/Raw Data/Income_Poverty-Census.csv', header= 3)
df_income = df_income[df_income['County FIPS Code']!=0]
df_income['County'] = df_income['Name'] + ', ' + df_income['Postal Code']
df_income = df_income.rename(columns = {'Poverty Percent, All Ages': 'Poverty (%)','Poverty Percent, Age 0-17': 'Poverty (%) U18'})
df_income = df_income[['County','Poverty (%)','Poverty (%) U18']]
df_income

,County,Poverty (%),Poverty (%) U18
2,"Autauga County, AL",11.8,15.7
3,"Baldwin County, AL",12.4,16.1
4,"Barbour County, AL",26.7,37.7
5,"Bibb County, AL",20.0,25.5
6,"Blount County, AL",13.6,15.8
...,...,...,...
3191,"Sweetwater County, WY",12.4,14.9
3192,"Teton County, WY",5.2,5.3
3193,"Uinta County, WY",10.8,13.2
3194,"Washakie County, WY",10.9,14.3


Before mergine our Census Data and CDC data together we should update some mismatching county names. I decided to follow the format that will allow us to easily merge with the Population Density Data Later. Looking at our merge we can see the Connecticut Census does not use Counties, but rather developmental zones. While this is not ideal, we can use the State average to fill that data in. 

In [22]:
df_combined['County'] = df_combined['County'].replace({'Petersburg Borough/Census Area, AK': 'Petersburg Borough, AK',
                                  'Wade Hampton Census Area, AK':'Kusilvak Census Area, AK',
                                  'Shannon County, SD' : 'Oglala Lakota County, SD'})

df_income['County'] = df_income['County'].replace({'La Salle County, IL': 'LaSalle County, IL'})

In [23]:
df_combined = df_income.merge(df_combined, on = 'County',how = 'right')
df_combined[df_combined['Poverty (%)'].isna()]

,County,Poverty (%),Poverty (%) U18,Drugs,Alcohol,Firearms,All,Homicides,Population
224,"Fairfield County, CT",NaN,NaN,1582,670,230,862,325,10333637
225,"Hartford County, CT",NaN,NaN,2511,895,262,1019,412,9839496
226,"Litchfield County, CT",NaN,NaN,556,238,103,282,30,2027568
227,"Middlesex County, CT",NaN,NaN,464,216,70,237,28,1805403
228,"New Haven County, CT",NaN,NaN,2535,838,290,996,405,9451072
229,"New London County, CT",NaN,NaN,853,341,133,395,65,2977239
230,"Tolland County, CT",NaN,NaN,329,129,74,214,18,1665722
231,"Windham County, CT",NaN,NaN,377,143,64,196,19,1288253


In [25]:
df_income = pd.read_csv('data/Raw Data/Income_Poverty-Census.csv', header= 3)
df_combined.loc[df_combined['Poverty (%)'].isna(),'Poverty (%)'] = df_income.loc[df_income['Name'] == 'Connecticut','Poverty Percent, All Ages'].mean()
df_combined.loc[df_combined['Poverty (%) U18'].isna(),'Poverty (%) U18'] = df_income.loc[df_income['Name'] == 'Connecticut','Poverty Percent, Age 0-17'].mean()

## Population Density 

Finally, we can bring in our population density data (also from the US Census). This dataset proved to be the most difficult to fold into our combined dataframe, however most of the step just involve massaging our County name into the proper format (changing county names, adding state abbreviations, etc.)

In [29]:
df_pop_density = pd.read_csv('data/Raw Data/Average_Household_Size_and_Population_Density-Census.csv')
df_pop_density 

,OBJECTID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,...,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
0,1,161526,1001,1.539602e+09,25706961,Autauga County,Alabama,2.59,0.05,2.59,...,55200,NaN,35.853419,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.066037,0.150256
1,2,161527,1003,4.117547e+09,1133055836,Baldwin County,Alabama,2.61,0.04,2.66,...,208107,NaN,50.541504,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,4.483746,0.409904
2,3,161528,1005,2.292145e+09,50538698,Barbour County,Alabama,2.49,0.07,2.44,...,25782,NaN,11.247981,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.695262,0.223270
3,4,161529,1007,1.612167e+09,9602089,Bibb County,Alabama,2.99,0.14,3.05,...,22527,NaN,13.973114,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,1.887514,0.156473
4,5,161530,1009,1.670104e+09,15015423,Blount County,Alabama,2.77,0.05,2.85,...,57645,NaN,34.515816,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.423552,0.164405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,3216,1804553,72145,1.187776e+08,57795019,Vega Baja Municipio,Puerto Rico,2.93,0.09,2.93,...,53371,NaN,449.335380,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.485650,0.010381
3216,3217,1804554,72147,1.315414e+08,552192819,Vieques Municipio,Puerto Rico,3.55,0.38,3.03,...,8771,NaN,66.678630,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.675819,0.012181
3217,3218,1804555,72149,9.229857e+07,3622639,Villalba Municipio,Puerto Rico,2.96,0.10,3.09,...,22993,NaN,249.115455,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.431598,0.008189
3218,3219,1804556,72151,1.430052e+08,72592521,Yabucoa Municipio,Puerto Rico,2.91,0.10,2.90,...,34149,NaN,238.795547,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.676628,0.012229


In [30]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

df_pop_density['State'] = df_pop_density['State'].replace(us_state_to_abbrev)
df_pop_density['County'] = df_pop_density['NAME'] + ', ' + df_pop_density['State']
df_pop_density = df_pop_density.rename(columns = {'B01001_calc_PopDensity': 'Pop. Den.'})[['County','State','Pop. Den.']]

df_pop_density

,County,State,Pop. Den.
0,"Autauga County, AL",AL,35.853419
1,"Baldwin County, AL",AL,50.541504
2,"Barbour County, AL",AL,11.247981
3,"Bibb County, AL",AL,13.973114
4,"Blount County, AL",AL,34.515816
...,...,...,...
3215,"Vega Baja Municipio, PR",PR,449.335380
3216,"Vieques Municipio, PR",PR,66.678630
3217,"Villalba Municipio, PR",PR,249.115455
3218,"Yabucoa Municipio, PR",PR,238.795547


In [31]:

df_pop_density['County'] = df_pop_density['County'].replace({'Doña Ana County, NM': 'Dona Ana County, NM',
                                                             'DeKalb County, IN': 'De Kalb County, IN',
                                                             'LaPorte County, IN': 'La Porte County, IN',
                                                             'Anchorage Municipality, AK' : 'Anchorage Borough, AK',
                                                             'Juneau City and Borough, AK':'Juneau Borough, AK',
                                                             'Yakutat City and Borough, AK' : 'Yakutat Borough, AK'})

df_combined = df_pop_density.merge(df_combined, on = 'County', how = 'right')
df_combined.head()


,County,State,Pop. Den.,Poverty (%),Poverty (%) U18,Drugs,Alcohol,Firearms,All,Homicides,Population
0,"Autauga County, AL",AL,35.853419,11.8,15.7,61,33,78,115,32,609875
1,"Baldwin County, AL",AL,50.541504,12.4,16.1,377,237,281,444,68,2250866
2,"Barbour County, AL",AL,11.247981,26.7,37.7,17,14,30,41,29,287620
3,"Bibb County, AL",AL,13.973114,20.0,25.5,47,Suppressed,29,38,21,248120
4,"Blount County, AL",AL,34.515816,13.6,15.8,140,48,83,122,38,635351


Above we can see what our consolidated dataset looks like. From here we can save it and continue on to the EDA portion of our analysis. Please join me in part 2 of our notebook. 

In [32]:
df_combined.to_csv('data/Homicide_Study_data.csv',index = False)